# Baseline Solution: Fake or Real - The Impostor Hunt in Texts 🔍

---

Here we provide the baseline solution for the *Fake or Real: The Impostor Hunt in Texts* challenge!
In this notebook, we walk you through two **simple, interpretable, and ML-free approaches** to tackle the problem of detecting fake texts.

### 💡 The overview of first approach:

We use the `langdetect` library to analyze each text by identifying the presence of **English vs. non-English words**. Here's the idea:

1. **Detect Language**: We break the text into words and determine the language of each.
2. **Calculate Proportion**: We then compute the percentage of English words in the entire text.
3. **Assign Label**: The text which gets higher percentage of English words is classified as **Real** and its number is saved to the results list.

---

### 📦 Getting Started: Install & Import Required Packages


In [ ]:
# !pip install langdetect
!pip install pandas
!pip install unicodedata2
!pip install scikit-learn

import os
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import unicodedata

import string
from sklearn.metrics import accuracy_score
import numpy as np
DetectorFactory.seed = 42

# The following is an enhanced markdown description for the cell above, formatted for direct inclusion as a Markdown cell in a Jupyter/Colab notebook.

"""
# 🚀 Data Ingestion Function: `read_texts_from_dir`

This cell defines the **`read_texts_from_dir`** function, a critical utility for **efficiently loading and structuring raw text data** from a hierarchical file system into a clean pandas DataFrame. It acts as the initial ETL (Extract, Transform, Load) step for downstream NLP tasks.

---

## 🎯 Purpose and Functionality

The core objective of this function is to automate the process of collecting text files that are presumably organized in pairs within subdirectories (e.g., `data/doc_001/file_a.txt`, `data/doc_001/file_b.txt`, etc.).

### Parameters
| Parameter | Type | Description |
| :--- | :--- | :--- |
| **`dir_path`** | `str` | The root path to the directory containing all subdirectories with the text files. |

### Process Flow
1.  **Directory Pre-count:** The function first calculates the total number of subdirectories (`dir_count`) within the provided `dir_path` using `os.walk` and `os.path.isdir`. This is a **performance optimization** to pre-allocate the data structure (`data=[0 for _ in range(dir_count)]`), ensuring efficient memory usage as the list is populated.
2.  **File Reading Loop:** The unselected portion of the code would then traverse these subdirectories, read the content of the relevant text files (likely two per subfolder), and create a list of records.
3.  **DataFrame Construction:** Finally, this list of records is transformed into a pandas DataFrame.

### Output Structure
The resulting DataFrame (`df`) is structured for comparative text analysis:

| Column | Data Type | Meaning |
| :--- | :--- | :--- |
| **`id`** | *Varies (Int/Str)* | A unique identifier, typically derived from the subdirectory name, serving as a primary key for the document pair. |
| **`file_1`** | `str` | The text content of the first document (e.g., the source text or prompt). |
| **`file_2`** | `str` | The text content of the second document (e.g., the potential duplicate, response, or comparison text). |

---

## 💡 Application and Relevance

Given the output format (`file_1` and `file_2`), this function is highly relevant for common NLP and data science problems like:
* **Plagiarism Detection:** Comparing a submitted document (`file_2`) against a source document (`file_1`). 🕵️
* **Semantic Similarity/Paraphrasing:** Evaluating how closely the meaning of two texts aligns. 🤝
* **Parallel Corpus Development:** Creating structured data for machine translation or text-to-text generation tasks. 📝
* **ML Data Preparation:** Creating the input features and labels required for training models.

In [ ]:
def read_texts_from_dir(dir_path):
  """
  Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

  Params:
    dir_path (str): path to the directory with data
  """
  # Count number of directories in the provided path
  dir_count = sum(os.path.isdir(os.path.join(root, d)) for root, dirs, _ in os.walk(dir_path) for d in dirs)
  data=[0 for _ in range(dir_count)]
  print(f"Number of directories: {dir_count}")

  # For each directory, read both file_1.txt and file_2.txt and save results to the list
  i=0
  for folder_name in sorted(os.listdir(dir_path)):
    folder_path = os.path.join(dir_path, folder_name)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, 'file_1.txt'), 'r', encoding='utf-8') as f1:
          text1 = f1.read().strip()
        with open(os.path.join(folder_path, 'file_2.txt'), 'r', encoding='utf-8') as f2:
          text2 = f2.read().strip()
        index = int(folder_name[-4:])
        data[i]=(index, text1, text2)
        i+=1
      except Exception as e:
        print(f"Error reading directory {folder_name}: {e}")

  # Change list with results into pandas DataFrame
  df = pd.DataFrame(data, columns=['id', 'file_1', 'file_2']).set_index('id')
  return df

In [ ]:
# Use the above function to load both train and test data
train_path="/home/samer/Desktop/competitions/Kaggle challenge Fake or Real: The Impostor Hunt in Texts/fake-or-real-the-impostor-hunt/data/train"
df_train=read_texts_from_dir(train_path)
test_path="/home/samer/Desktop/competitions/Kaggle challenge Fake or Real: The Impostor Hunt in Texts/fake-or-real-the-impostor-hunt/data/test"
df_test=read_texts_from_dir(test_path)

Number of directories: 95
Number of directories: 1068


In [ ]:
df_train.head()

In [ ]:
df_test.head()

---

### 🏷️ Read the Labels

Next, we’ll load the **labels** associated with each text sample.
These labels indicate which text is **Real** - 1 or 2. The labels will serve as our ground truth for evaluation.

We’ll again use `Pandas` to read the label file into a DataFrame and inspect its structure.


In [ ]:
# Load ground truth for train data
df_train_gt=pd.read_csv("/home/samer/Desktop/competitions/Kaggle challenge Fake or Real: The Impostor Hunt in Texts/fake-or-real-the-impostor-hunt/data/train.csv")
df_train_gt

---
# ✍️ Baseline Prediction Method: English Word Percentage

This cell defines a simple, language-based baseline method for the competition.

### 📜 Method Description
The `baseline_method_english_word` function predicts which text is "Real" (or more authentic/less generated) by comparing the **percentage of recognized English words** in `text_1` and `text_2`.

-   It iterates through the rows of the input DataFrame.
-   For each pair of texts, it calculates a "score" based on the proportion of English words found in it.
-   It predicts `1` if the score for `text_1` is higher (more English words) and `2` if the score for `text_2` is higher.

### 🎯 Expected Use Case
This serves as a quick and easily reproducible **initial benchmark** to gauge the difficulty of the problem before developing more sophisticated Machine Learning models.

In [ ]:
def baseline_method_english_word(df):
  """
  This baseline method predicts which of the texts is Real, based on the percentage of English words in each text.
  It returns list with predictions.

  Params:
    df (pd.DataFrame): dataframe with all texts
  """
  # Create lists in which scores will be saved for file_1 (left_scores) and files_2 (right_scores)
  left_scores=[0 for _ in range(df.shape[0])]
  right_scores=[0 for _ in range(df.shape[0])]
  # For each row in the DataFrame and for each element of this row run the algorithm for detecting English words
  for j in range(df.shape[0]):
    for z in range(df.shape[1]):
      sum_english=0
      n=10
      delete=str.maketrans('', '', string.punctuation+'\n')
      cleaned=df.iloc[j].iloc[z].translate(delete)
      text_to_check=cleaned.split(" ")
      text_to_check=[' '.join(text_to_check[i:i+n]) for i in range(0, len(text_to_check),n)]

      # Run algorithm for detecting English words
      for i in range(len(text_to_check)):
        try:
          language=detect(text_to_check[i])
        except LangDetectException as e:
          pass
        if language=='en':
          sum_english+=1
      result=sum_english/len(text_to_check)
      if z==0:
        left_scores[j]=result
      elif z==1:
        right_scores[j]=result
      else:
        print('Wrong')
  # Create list with predictions by setting value in list to 1 if the first text is `Real` or 2 when the second seems to be better
  predictions=[1 if left_scores[k]>right_scores[k] else 2 for k in range(len(left_scores))]
  return predictions

In [ ]:
def evaluate_baseline(predictions, gt_list, text='Score with english detection:'):
  """
  Evaluates the predictions for train data, when the ground truth is provided.

  Params:
    predictions (list): list of predictions
    gt_list (list): list of predictions
    text (str): text to be printed together with the result
  """
  acc_score = accuracy_score(gt_list, predictions)
  print(text,acc_score)

---
#### 📊 Results

In [ ]:
# Use the algorithm for the train data and check accuracy
predictions_train=baseline_method_english_word(df_train)
gt_train=list(df_train_gt['real_text_id'])
evaluate_baseline(predictions_train, gt_train)

Score with english detection: 0.6631578947368421


In [ ]:
# Use the algorithm for the test data
predictions_test=baseline_method_english_word(df_test)

### Prepare format for sample solution

In [ ]:
# Change the format of predictions into requested format, as described in Overview section of this competition
df_results_test=pd.DataFrame(predictions_test)
output_df = df_results_test.copy()
output_df.columns = ['real_text_id']
output_df.reset_index(inplace=True)
output_df.rename(columns={'index': 'id'}, inplace=True)
output_df

In [ ]:
output_df.to_csv('sample_submission_1.csv', index=False)

---

### 🔡 Character-Level Baseline

In addition to analyzing words, we can explore a **character-level approach** as an alternative baseline.

This method evaluates the **proportion of Latin characters** in the text, instead of relying on word-based language detection.

By comparing the ratio of English characters to total characters, we generate another set of predictions—offering a complementary perspective to our word-level strategy.

In [ ]:
def is_latin_char(char):
  """
  Detect if given character is from Latin alphabet.

  Params:
    char (str): given character
  """
  char=str(char)
  try:
    name=unicodedata.name(char)
    return 'LATIN' in name
  except ValueError:
    return False

def baseline_chars_method(df):
  """
  This baseline method predicts which of the texts is Real, based on the percentage of Lating letters in each text.
  It returns list with predictions.

  Params:
    df (pd.DataFrame): dataframe with all texts
  """
  # Create lists in which scores will be saved for file_1 (left_scores) and files_2 (right_scores)
  left_scores=[0 for _ in range(df.shape[0])]
  right_scores=[0 for _ in range(df.shape[0])]
  # For each row in the DataFrame and for each element of this row run the algorithm for detecting Latin chars
  for j in range(df.shape[0]):
    for z in range(df.shape[1]):
      sum_latin=0
      count_spaces=0
      delete=str.maketrans('', '', string.punctuation+'\n')
      cleaned=df.iloc[j].iloc[z].translate(delete)

      # Run algorithm for detecting Latin chars
      for i in range(len(cleaned)):
        if cleaned[i] !=' ':
          if is_latin_char(cleaned[i]):
            sum_latin+=1
        else:
          count_spaces+=1
      if len(cleaned)==0:
        result=0
      else:
        result=sum_latin/(len(cleaned)-count_spaces)
      if z==0:
        left_scores[j]=result
      elif z==1:
        right_scores[j]=result
      else:
        print('Wrong')
  # Create list with predictions by setting value in list to 1 if the first text is `Real` or 2 when the second seems to be better
  predictions=[1 if left_scores[k]>right_scores[k] else 2 for k in range(len(left_scores))]
  return predictions

---
#### 📊 Results

In [ ]:
# Use the algorithm for the train data and check accuracy
predictions_train_char=baseline_chars_method(df_train)
gt_train=list(df_train_gt['real_text_id'])
evaluate_baseline(predictions_train_char, gt_train, text='Score with latin detection:')

Score with latin detection: 0.5368421052631579


In [ ]:
# Use the algorithm for the test data
preds_test_char=baseline_chars_method(df_test)

### Prepare format for sample solution

In [ ]:
# Change the format of predictions into requested format, as described in Overview section of this competition
df_results_test_char=pd.DataFrame(preds_test_char)
output_df_char = df_results_test_char.copy()
output_df_char.columns = ['real_text_id']
output_df_char.reset_index(inplace=True)
output_df_char.rename(columns={'index': 'id'}, inplace=True)
output_df_char

In [ ]:
output_df_char.to_csv('sample_submission_2.csv', index=False)

---
# **Submitted solution generation** 

## 🐍 Setup, Imports, and Environment Configuration ⚙️

This cell performs critical **setup tasks** for the notebook's execution.

1.  **Library Imports:** Essential libraries are imported for data manipulation (`pandas`), system interaction (`os`), Google's Generative AI services (`google-genai`), and model evaluation (`sklearn.metrics`).
2.  **API Key Loading:** The `dotenv` library is used with `load_dotenv` to securely load necessary environment variables, typically including the **Gemini API Key**, from a `.env` file for access by the `google.genai` client.
3.  **Utility & Timing:** The `time` module is imported, likely for benchmarking or tracking the duration of computationally intensive tasks.
4.  **Submission File Preparation (Inferred):** The trailing lines of code strongly suggest the final steps of a **data science pipeline**.
    * A DataFrame (`output_df_char`) containing predictions or processed results is prepared.
    * It's reset and renamed to include an **'id' column**.
    * It is then saved to a CSV file (`submission_5.csv`), which is the standard format for submitting predictions to platforms like **Kaggle**.

In [ ]:
import pandas as pd
import os
from google import genai
import google.genai as genai
from google.genai import types
from sklearn.metrics import accuracy_score
import time
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()


def get_response(contents_1):
    """receive a string of the prompt for the model and
    returns a string of the article

    Args:
        contents_1 (str): The prompt to send to the generative model.
    """
    # Define the model name to use
    model_name = "gemma-3-27b-it" # Corrected model name based on traceback
    # Initialize the client with the API key
    client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
    x = True
    # Loop to handle potential API errors and retry
    while x == True:
        try:
            # Generate content using the specified model and prompt
            response_1 = client.models.generate_content(
                model=model_name,
                contents=contents_1,
                )
            # Strip whitespace from the response text
            result_1 = response_1.text.strip()
            x = False
            return result_1
        except:
            # If an error occurs, wait for 30 seconds before retrying
            time.sleep(30)
            continue

    return result_1

def process_articles(df: pd.DataFrame):
    """
    Iterates over a DataFrame, reading file paths from 'file_1' and 'file_2'
    columns, and prints their contents.

    Args:
        df (pd.DataFrame): DataFrame with 'file_1' and 'file_2' columns
                           containing file paths.
    """
    print("--- Starting to process articles ---")
    predictions = []
    counter = 0
    results = ["Article 1", "Article 2"]
    turns = 0
    # itertuples() is generally more performant than iterrows()
    for row in df.itertuples(index=False):
        # Using getattr to access columns by name from the named tuple
        print("this is row", row)
        f1 = getattr(row, 'file_1')
        f2 = getattr(row, 'file_2')


        # Define the model name and initialize the client
        model_name = "gemma-3-27b-it" # Corrected model name based on traceback
        client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))


        # models = client.models.list()
        # for m in models:
        #     print(m)
        # break

        # Create the prompt for the generative model
        contents_1 = f"""
                        Article 1: {f1}
                        Article 2: {f2}

                        Only one of the articles above is real, and the other is fake.
                        Let's think step by step. First, analyze both articles to find signs of being fake or real. Second, compare them to determine which one is more likely to be real. Finally, state your conclusion.

                        take into consideration that only texts in english language can be real and all other languages are fake.

                        regardless of anything do not include the thinking steps in the output and return only one word referring to the real article [Article 1 or Article 2].

                        Pay high attention please because it matters a lot.
                        """


        try:
            # Generate content from the model
            response_1 = client.models.generate_content(
                model=model_name,
                contents=contents_1,
                )

        except:
            # Wait for 30 seconds if there's an API error
            time.sleep(30)
        # Get the text from the response
        result_1 = response_1.text.strip()
        counter+=1
        print(f"Processed row {counter}: {result_1}")

        # Convert the model's text response to a prediction (1 or 2)
        if result_1 == "Article 1":
            predict = 1
            print("predict", predict)
            predictions.append(predict)
        elif result_1 == "Article 2":
            predict = 2
            print("predict", predict)
            predictions.append(predict)
        else:
            # If the response is not as expected, retry up to 10 times
            while result_1 not in results and turns < 10:
                result_1 = get_response(contents_1)
                print(f"Processed row {counter}: {result_1}")
                turns += 1
            else:
                # Process the result from retries
                if result_1 == "Article 1":
                    predict = 1
                    print("predict", predict)
                    predictions.append(predict)
                elif result_1 == "Article 2":
                    predict = 2
                    print("predict", predict)
                    predictions.append(predict)
                else:
                    # Default to 1 if still no valid response
                    predict = 1
                    print("predict", predict)
                    predictions.append(predict)



    print("Predictions:", predictions)
    # Load ground truth labels for evaluation
    df_train_gt=pd.read_csv(os.getenv("df_train_gt"))
    df_train_gt
    y_true = [int(row["real_text_id"]) for index, row in df_train_gt.iterrows()]
    print("y_true", y_true)

    # Calculate and print the accuracy score
    # acc = accuracy_score(y_true, predictions)
    # print("Accuracy for the model: ", acc)

    return predictions

# This block runs when the script is executed directly
if __name__ == '__main__':
    # Define paths for train and test data
    train_path=os.getenv("train_path")
    # Load training data
    df_train=read_texts_from_dir(train_path)
    test_path=os.getenv("test_path")
    # Load test data
    df_test=read_texts_from_dir(test_path)

    # Process the training data to get predictions
    predictions = process_articles(df_test)
    # Create a DataFrame from the predictions
    df_results_test_char=pd.DataFrame(predictions)
    output_df_char = df_results_test_char.copy()
    output_df_char.columns = ['real_text_id']
    output_df_char.reset_index(inplace=True)
    output_df_char.rename(columns={'index': 'id'}, inplace=True)
    output_df_char
    # Save the results to a CSV file (commented out)
    output_df_char.to_csv('submission_5.csv', index=False)